In [11]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob
from sklearn.model_selection import KFold
DATA_DIR = '/kaggle/input'

# Directory to save logs and trained model
ROOT_DIR = '/kaggle/working'

# **Встановлення моделі та всіх необхідних конфігів.**
<p>Це все треба один раз запустити і потім це треба закоментити</p>

In [12]:
!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
!ls -lh mask_rcnn_coco.h5

!pip install gdown
import gdown


url = 'https://drive.google.com/uc?id='+'1ZKSpyTya52QLb-ufq2rjbyyHoeuUn2gx'
output = '/kaggle/working/model.h5'
gdown.download(url, output, quiet=False)

#!python setup.py -q install

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 137.67 MiB | 48.36 MiB/s, done.
Resolving deltas: 100% (558/558), done.
-rw-r--r-- 1 root root 246M Dec  6  2021 mask_rcnn_coco.h5


Downloading...
From (original): https://drive.google.com/uc?id=1ZKSpyTya52QLb-ufq2rjbyyHoeuUn2gx
From (redirected): https://drive.google.com/uc?id=1ZKSpyTya52QLb-ufq2rjbyyHoeuUn2gx&confirm=t&uuid=2f0b5127-152b-45bb-8896-6cf9b0d14214
To: /kaggle/working/model.h5
100%|██████████| 179M/179M [00:01<00:00, 175MB/s]  


'/kaggle/working/model.h5'

In [13]:
# Import Mask RCNN
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [14]:
test_dicom_dir = os.path.join(DATA_DIR, 'stage_2_test_images')

COCO_WEIGHTS_PATH = "mask_rcnn_coco.h5"

# **Конфігурація моделі перед запуском**

In [15]:
def get_dicom_fps(dicom_dir):
    dicom_fps = glob.glob(dicom_dir+'/'+'*.dcm')
    return list(set(dicom_fps))

def parse_dataset(dicom_dir, anns): 
    image_fps = get_dicom_fps(dicom_dir)
    image_annotations = {fp: [] for fp in image_fps}
    for index, row in anns.iterrows(): 
        fp = os.path.join(dicom_dir, row['patientId']+'.dcm')
        image_annotations[fp].append(row)
    return image_fps, image_annotations 


class DetectorConfig(Config):
    """
    Конфігурація основних параметрів
    """ 
    NAME = 'pneumonia'
    GPU_COUNT = 1
    
    BACKBONE = 'resnet50'
    
    NUM_CLASSES = 2  
    
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256
    RPN_ANCHOR_SCALES = (16, 32, 64, 128)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 4
    DETECTION_MAX_INSTANCES = 3
    DETECTION_MIN_CONFIDENCE = 0.78  
    DETECTION_NMS_THRESHOLD = 0.01

    STEPS_PER_EPOCH = 200    
    
config = DetectorConfig() 
config.display()



class DetectorDataset(utils.Dataset):
    """
    Dataset class for training pneumonia detection on the RSNA pneumonia dataset.
    """

    def __init__(self, paths_to_images, annotations_per_image, image_height, image_width):
        # Виклик конструктора базового класу
        super().__init__()

        # Додавання класу "Легенева непрозорість" до набору даних
        self.add_class('pneumonia', 1, 'Lung Opacity')

        # Додавання інформації про зображення в набір даних
        for index, image_path in enumerate(paths_to_images):
            # Отримання анотацій для конкретного зображення
            current_annotations = annotations_per_image[image_path]
            # Додавання зображення та його анотацій в набір даних
            self.add_image(
                'pneumonia',  # Джерело даних
                image_id=index, 
                path=image_path,  
                annotations=current_annotations,  
                orig_height=image_height,  
                orig_width=image_width 
            )

    def image_reference(self, image_id):
        image_info = self.image_info[image_id]
        # Повертаємо шлях до зображення
        return image_info['path']

    def load_image(self, image_id):
        # Отримання інформації про зображення за ID
        image_info = self.image_info[image_id]
        # Шлях до файлу зображення
        image_path = image_info['path']
        # Завантаження DICOM файла
        dicom_file = pydicom.read_file(image_path)
        # Перетворення DICOM у numpy масив
        image_array = dicom_file.pixel_array
        # Якщо зображення в градаціях сірого, конвертуємо у RGB для уніфікації
        if len(image_array.shape) != 3 or image_array.shape[2] != 3:
            image_array = np.stack((image_array,) * 3, axis=-1)
        # Повертаємо зображення
        return image_array

    def load_mask(self, image_id):
        # Отримання детальної інформації про зображення за його ідентифікатором
        image_details = self.image_info[image_id]
        # Витягуємо анотації, пов'язані з цим зображенням
        image_annotations = image_details['annotations']
        # Кількість анотацій
        annotations_count = len(image_annotations)
        # Ініціалізація маски та масиву ідентифікаторів класів
        if annotations_count == 0:
            # Створення порожньої маски, якщо анотацій немає
            mask = np.zeros((image_details['orig_height'], image_details['orig_width'], 1), dtype=np.uint8)
            class_ids = np.zeros((1,), dtype=np.int32)
        else:
            # Створення маски з анотаціями
            mask = np.zeros((image_details['orig_height'], image_details['orig_width'], annotations_count), dtype=np.uint8)
            class_ids = np.zeros((annotations_count,), dtype=np.int32)
            for idx, annotation in enumerate(image_annotations):
                if annotation['Target'] == 1:
                    # Координати об'єкта анотації
                    rect_start_x = int(annotation['x'])
                    rect_start_y = int(annotation['y'])
                    rect_width = int(annotation['width'])
                    rect_height = int(annotation['height'])
                    # Створення індивідуальної маски для анотації
                    individual_mask = mask[:, :, idx].copy()
                    cv2.rectangle(individual_mask, (rect_start_x, rect_start_y), (rect_start_x + rect_width, rect_start_y + rect_height), 255, -1)
                    mask[:, :, idx] = individual_mask
                    class_ids[idx] = 1
        # Конвертація типу маски для використання у подальшій обробці
        return mask.astype(np.bool_), class_ids.astype(np.int32)
    
    
# Зробити прогнози на тестових зображеннях, записати зразок подання
def predict(image_fps, filepath=ROOT_DIR+'/submission.csv', min_conf=0.98): 
    # припускаємо квадратне зображення   
    with open(filepath, 'w') as file:
        file.write('patientId,PredictionString'+"\n")        
        for image_id in tqdm(image_fps): 
            ds = pydicom.read_file(image_id)
            image = ds.pixel_array
        
            # Якщо в чорно-білому. Конвертуємо в RGB для послідовності
            if len(image.shape) != 3 or image.shape[2] != 3:
                image = np.stack((image,) * 3, -1) 
            
            patient_id = os.path.splitext(os.path.basename(image_id))[0]

            results = model.detect([image])
            r = results[0]

            out_str = ""
            out_str += patient_id 
            assert( len(r['rois']) == len(r['class_ids']) == len(r['scores']) )
            if len(r['rois']) == 0:
                out_str += ","
            else: 
                num_instances = len(r['rois'])
                out_str += ","
                for i in range(num_instances): 
                    if r['scores'][i] > min_conf: 
                        out_str += ' '
                        out_str += str(round(r['scores'][i], 2))
                        out_str += ' '

                        # x1, y1, ширина, висота 
                        x1 = r['rois'][i][1]
                        y1 = r['rois'][i][0]
                        width = r['rois'][i][3] - x1 
                        height = r['rois'][i][2] - y1 
                        bboxes_str = "{} {} {} {}".format(x1, y1, \
                                                          width, height)    
                        out_str += bboxes_str
            file.write(out_str+"\n")


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        3
DETECTION_MIN_CONFIDENCE       0.78
DETECTION_NMS_THRESHOLD        0.01
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE              

# **Ініціалізація моделі**

In [16]:
class InferenceConfig(DetectorConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# перестворення моделі
model = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=ROOT_DIR)


#завантаження заздалегідь натренованої моделі, якщо закоментувати model_path, то завантажиться поточна модель
model_path = "/kaggle/working/model.h5"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  /kaggle/working/model.h5


# **Обробка тестових даних**

In [17]:

test_image_fps = get_dicom_fps(test_dicom_dir)
sample_submission_fp = ROOT_DIR+'/submission.csv'
predict(test_image_fps, filepath=sample_submission_fp)
#output = pd.read_csv(sample_submission_fp)
# output.head(50)

100%|██████████| 3000/3000 [07:35<00:00,  6.91it/s]
